In [1]:
import mysql.connector
from mysql.connector import Error

In [8]:
!mysql.server start

Starting MySQL
 SUCCESS! 


In [2]:
def check_connection(host, db_name, user, psswd):
    try:
        connection = mysql.connector.connect(host=host,
                                             database=db_name,
                                             user=user,
                                             password=psswd)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)

    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [3]:
host = 'localhost'
db_name = 'sports_booking'
user = 'root'
psswd = ''
check_connection(host, db_name, user, psswd)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('sports_booking',)
MySQL connection is closed


In [15]:
my_executor = testito(host, db_name,user,psswd)

In [4]:
def execute_sql_wrapper(host, db_name, user, psswd):
    host = host
    db_name = db_name
    user = user
    psswd = psswd
    
    def execute_sql(query):
        try:
            connection = mysql.connector.connect(host=host,
                                                 database=db_name,
                                                 user=user,
                                                 password=psswd)
            cursor = connection.cursor()
            result = cursor.execute(query)
            connection.commit()
        except mysql.connector.Error as error:
            print("Failed to create table in MySQL: {}".format(error))
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")

    
    return execute_sql
    

def create_table_wrapper(host, db_name, user, psswd):
    def create_table(query):
        try:
            connection = mysql.connector.connect(host=host,
                                                 database=db_name,
                                                 user=user,
                                                 password=psswd)
            cursor = connection.cursor()
            result = cursor.execute(query)
            print(result)

        except mysql.connector.Error as error:
            print("Failed to create table in MySQL: {}".format(error))
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")
    return create_table
            
def show_tables_wrapper(host, db_name, user, psswd):
    def show_tables():
        try:
            connection = mysql.connector.connect(host=host,database=db_name,user=user,password=psswd)
            cursor = connection.cursor()
            cursor.execute("Show tables;")
            result = cursor.fetchall()
            for table in result:
                print(table)

        except mysql.connector.Error as error:
            print("Failed to create table in MySQL: {}".format(error))
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")
    return show_tables

def print_table_wrapper(host, db_name, user, psswd):
    def print_table(query):
        try:
            connection = mysql.connector.connect(host=host,
                                                 database=db_name,
                                                 user=user,
                                                 password=psswd)
            cursor = connection.cursor()
            cursor.execute(query)
            result = cursor.fetchall()
            for row in result:
                print(row)

        except mysql.connector.Error as error:
            print("Failed to create table in MySQL: {}".format(error))
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")
    return print_table

def insert_values_wrapper(host, db_name, user, psswd):
    def insert_values(values):
        try:
            connection = mysql.connector.connect(host=host,
                                                 database=db_name,
                                                 user=user,
                                                 password=psswd)
            cursor = connection.cursor()
            for value in values:
                cursor.execute(value)
            connection.commit()
        except mysql.connector.Error as error:
            print("Failed to create table in MySQL: {}".format(error))
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")
    return insert_values


In [5]:
create_table = create_table_wrapper(host, db_name, user, psswd)
show_tables = show_tables_wrapper(host, db_name, user, psswd)
insert_values = insert_values_wrapper(host, db_name, user, psswd)
execute_sql = execute_sql_wrapper(host, db_name, user, psswd)
print_table = print_table_wrapper(host, db_name, user, psswd)

In [6]:
show_tables()

('bookings',)
('member_bookings',)
('members',)
('pending_terminations',)
('rooms',)
MySQL connection is closed


## Creating the tables

In [28]:
create_members = """
CREATE TABLE members (
    id VARCHAR(255) PRIMARY KEY,
    password VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    member_since TIMESTAMP NOT NULL DEFAULT NOW(),
    payment_due FLOAT(4,2) NOT NULL DEFAULT 0
);
"""
create_table(create_members)

Failed to create table in MySQL: 1050 (42S01): Table 'members' already exists
MySQL connection is closed


In [33]:
show_tables()

('bookings',)
('members',)
('pending_terminations',)
('rooms',)
MySQL connection is closed


In [45]:
create_pending_terminations = """
CREATE TABLE pending_terminations(
    id VARCHAR(255) PRIMARY KEY,
    email VARCHAR(255) NOT NULL,
    request_date TIMESTAMP NOT NULL DEFAULT NOW(),
    payment_due FLOAT(4,2) NOT NULL DEFAULT 0
    
);
"""
create_table(create_pending_terminations)

None
MySQL connection is closed


In [32]:
show_tables()

('bookings',)
('members',)
('pending_terminations',)
('rooms',)
MySQL connection is closed


In [65]:
create_rooms = """
CREATE TABLE rooms(
    id VARCHAR(255) PRIMARY KEY,
    room_type VARCHAR(255) NOT NULL,
    price FLOAT(5,2) NOT NULL    
);
"""
create_table(create_rooms)

None
MySQL connection is closed


In [66]:
show_tables(host, db_name, user, psswd)

('members',)
('pending_terminations',)
('rooms',)
MySQL connection is closed


In [67]:
create_bookings = """
CREATE TABLE bookings(
    id INT PRIMARY KEY AUTO_INCREMENT,
    room_id VARCHAR(255) NOT NULL,
    booked_date DATE NOT NULL,
    booked_time TIME NOT NULL,
    member_id VARCHAR(255) NOT NULL,
    datetime_of_booking TIMESTAMP NOT NULL DEFAULT NOW(),
    payment_status VARCHAR(255) NOT NULL DEFAULT 'Unpaid',
    CONSTRAINT uc1 UNIQUE(room_id, booked_date, booked_time)
);
"""
create_table(create_bookings)

None
MySQL connection is closed


In [68]:
show_tables(host, db_name, user, psswd)

('bookings',)
('members',)
('pending_terminations',)
('rooms',)
MySQL connection is closed


In [69]:
alter_bookings = """ALTER TABLE bookings 
    ADD CONSTRAINT fk1 FOREIGN KEY(member_id) REFERENCES members(id) ON DELETE CASCADE ON UPDATE CASCADE,
    ADD CONSTRAINT fk2 FOREIGN KEY(room_id) REFERENCES rooms(id) ON DELETE CASCADE ON UPDATE CASCADE;"""
create_table(alter_bookings)

None
MySQL connection is closed


## Inserting Data 

### Populating members table

In [52]:
members_data = [('afeil', 'feil1988<3', 'Abdul.Feil@hotmail.com', '2017-04-15 12:10:13', 0),('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', '2018-02-06 16:48:43', 0), ('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', '2017-12-28 05:36:50', 0), ('little31', 'whocares31', 'Anthony_Little31@gmail.com', '2017-06-01 21:12:11', 10), ('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', '2017-05-30 17:30:22', 0), ('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', '2017-09-09 02:30:49', 10), ('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', '2018-01-09 17:36:49', 0), ('noah51', '18Oct1976#51', 'Noah51@gmail.com', '2017-12-16 22:59:46', 0),
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', '2017-10-12 05:39:20', 0), ('wyattgreat', 'wyatt111', 'Wyatt_Wisozk2@gmail.com', '2017-07-18 16:28:35', 0)]

In [53]:
insert_members = """INSERT INTO members (id, password, email, member_since, payment_due) VALUES ("{}","{}","{}","{}",{})"""
insert_statements = [insert_members.format(member[0], member[1], member[2], member[3], member[4]) for member in members_data]

In [54]:
insert_values(insert_statements, host, db_name, user, psswd)

MySQL connection is closed


In [55]:
print_table("members", host, db_name, user, psswd)

('afeil', 'feil1988<3', 'Abdul.Feil@hotmail.com', datetime.datetime(2017, 4, 15, 12, 10, 13), 0.0)
('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', datetime.datetime(2018, 2, 6, 16, 48, 43), 0.0)
('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', datetime.datetime(2017, 12, 28, 5, 36, 50), 0.0)
('little31', 'whocares31', 'Anthony_Little31@gmail.com', datetime.datetime(2017, 6, 1, 21, 12, 11), 10.0)
('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', datetime.datetime(2017, 5, 30, 17, 30, 22), 0.0)
('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', datetime.datetime(2017, 9, 9, 2, 30, 49), 10.0)
('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', datetime.datetime(2018, 1, 9, 17, 36, 49), 0.0)
('noah51', '18Oct1976#51', 'Noah51@gmail.com', datetime.datetime(2017, 12, 16, 22, 59, 46), 0.0)
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', datetime.datetime(2017, 10, 12, 5, 39, 20), 0.0)
('wyattgreat', 'wyatt111', 'Wyatt_Wisozk2@gmail.com', dateti

### Populating rooms table

In [56]:
rooms_data = [('AR', 'Archery Range', 120),('B1', 'Badminton Court', 8),('B2', 'Badminton Court', 8),
              ('MPF1', 'Multi Purpose Field', 50), ('MPF2', 'Multi Purpose Field', 60), ('T1', 'Tennis Court', 10),
              ('T2', 'Tennis Court', 10)]

In [62]:
insert_rooms = """INSERT INTO rooms (id, room_type, price) VALUES ("{}","{}",{})"""
insert_rooms = [insert_rooms.format(member[0], member[1], member[2]) for member in rooms_data]

In [70]:
insert_values(insert_rooms, host, db_name, user, psswd)

MySQL connection is closed


In [71]:
print_table('rooms', host, db_name, user, psswd)

('AR', 'Archery Range', 120.0)
('B1', 'Badminton Court', 8.0)
('B2', 'Badminton Court', 8.0)
('MPF1', 'Multi Purpose Field', 50.0)
('MPF2', 'Multi Purpose Field', 60.0)
('T1', 'Tennis Court', 10.0)
('T2', 'Tennis Court', 10.0)
MySQL connection is closed


### Populating bookings table

In [72]:
bookings_data = [(1, 'AR', '2017-12-26', '13:00:00', 'oreillys', '2017-12-20 20:31:27', 'Paid'),
(2, 'MPF1', '2017-12-30', '17:00:00', 'noah51', '2017-12-22 05:22:10', 'Paid'),
(3, 'T2', '2017-12-31', '16:00:00', 'macejkovic73', '2017-12-28 18:14:23', 'Paid'),
(4, 'T1', '2018-03-05', '08:00:00', 'little31', '2018-02-22 20:19:17', 'Unpaid'),
(5, 'MPF2', '2018-03-02', '11:00:00', 'marvin1', '2018-03-01 16:13:45', 'Paid'),
(6, 'B1', '2018-03-28', '16:00:00', 'marvin1', '2018-03-23 22:46:36', 'Paid'),
(7, 'B1', '2018-04-15', '14:00:00', 'macejkovic73', '2018-04-12 22:23:20', 'Cancelled'),
(8, 'T2', '2018-04-23', '13:00:00', 'macejkovic73', '2018-04-19 10:49:00', 'Cancelled'), 
(9, 'T1', '2018-05-25', '10:00:00', 'marvin1', '2018-05-21 11:20:46', 'Unpaid'),
(10, 'B2', '2018-06-12', '15:00:00', 'bbahringer', '2018-05-30 14:40:23', 'Paid')]

In [76]:
insert_bookings = """INSERT INTO bookings (room_id, booked_date, booked_time, member_id, datetime_of_booking, payment_status) VALUES ("{}","{}","{}","{}","{}","{}")"""
insert_bookings = [insert_bookings.format(member[1], member[2], member[3],member[4], member[5], member[6]) for member in bookings_data]

In [78]:
insert_values(insert_bookings, host, db_name, user, psswd)

MySQL connection is closed


In [79]:
print_table('bookings', host, db_name, user, psswd)

(1, 'AR', datetime.date(2017, 12, 26), datetime.timedelta(seconds=46800), 'oreillys', datetime.datetime(2017, 12, 20, 20, 31, 27), 'Paid')
(2, 'MPF1', datetime.date(2017, 12, 30), datetime.timedelta(seconds=61200), 'noah51', datetime.datetime(2017, 12, 22, 5, 22, 10), 'Paid')
(3, 'T2', datetime.date(2017, 12, 31), datetime.timedelta(seconds=57600), 'macejkovic73', datetime.datetime(2017, 12, 28, 18, 14, 23), 'Paid')
(4, 'T1', datetime.date(2018, 3, 5), datetime.timedelta(seconds=28800), 'little31', datetime.datetime(2018, 2, 22, 20, 19, 17), 'Unpaid')
(5, 'MPF2', datetime.date(2018, 3, 2), datetime.timedelta(seconds=39600), 'marvin1', datetime.datetime(2018, 3, 1, 16, 13, 45), 'Paid')
(6, 'B1', datetime.date(2018, 3, 28), datetime.timedelta(seconds=57600), 'marvin1', datetime.datetime(2018, 3, 23, 22, 46, 36), 'Paid')
(7, 'B1', datetime.date(2018, 4, 15), datetime.timedelta(seconds=50400), 'macejkovic73', datetime.datetime(2018, 4, 12, 22, 23, 20), 'Cancelled')
(8, 'T2', datetime.date(

## Views

In [56]:
create_view="""
CREATE VIEW member_bookings AS
SELECT bookings.id, room_id, room_type, booked_date, booked_time, member_id, datetime_of_booking, price, payment_status 
FROM
bookings
LEFT JOIN
rooms
ON
bookings.room_id = rooms.id;
"""
execute_sql(create_view)

MySQL connection is closed


In [57]:
print_table("SELECT * FROM member_bookings")

(1, 'AR', 'Archery Range', datetime.date(2017, 12, 26), datetime.timedelta(seconds=46800), 'oreillys', datetime.datetime(2017, 12, 20, 20, 31, 27), 120.0, 'Paid')
(2, 'MPF1', 'Multi Purpose Field', datetime.date(2017, 12, 30), datetime.timedelta(seconds=61200), 'noah51', datetime.datetime(2017, 12, 22, 5, 22, 10), 50.0, 'Paid')
(3, 'T2', 'Tennis Court', datetime.date(2017, 12, 31), datetime.timedelta(seconds=57600), 'macejkovic73', datetime.datetime(2017, 12, 28, 18, 14, 23), 10.0, 'Paid')
(4, 'T1', 'Tennis Court', datetime.date(2018, 3, 5), datetime.timedelta(seconds=28800), 'little31', datetime.datetime(2018, 2, 22, 20, 19, 17), 10.0, 'Unpaid')
(5, 'MPF2', 'Multi Purpose Field', datetime.date(2018, 3, 2), datetime.timedelta(seconds=39600), 'marvin1', datetime.datetime(2018, 3, 1, 16, 13, 45), 60.0, 'Paid')
(6, 'B1', 'Badminton Court', datetime.date(2018, 3, 28), datetime.timedelta(seconds=57600), 'marvin1', datetime.datetime(2018, 3, 23, 22, 46, 36), 8.0, 'Paid')
(7, 'B1', 'Badminton

## Stored Procedures

Note that when creating stored procedures through Python, it is not necessary (nor possible) to change the delimiter. More info here: https://stackoverflow.com/questions/48001322/mysql-error-syntax-in-python

In [64]:
create_insert_new_member = """
CREATE PROCEDURE insert_new_member(IN p_id VARCHAR(255), IN p_password VARCHAR(255), IN p_email VARCHAR(255))
BEGIN
    INSERT INTO members (id, password, email) VALUES (p_id, p_password, p_email);
END
"""

execute_sql(create_insert_new_member)

MySQL connection is closed


In [72]:
execute_sql("""CALL insert_new_member("testin01", "dlafjkgl", "ladfdskl@gmail.com")""")

MySQL connection is closed


In [73]:
print_table("SELECT * from members")

('afeil', 'feil1988<3', 'Abdul.Feil@hotmail.com', datetime.datetime(2017, 4, 15, 12, 10, 13), 0.0)
('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', datetime.datetime(2018, 2, 6, 16, 48, 43), 0.0)
('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', datetime.datetime(2017, 12, 28, 5, 36, 50), 0.0)
('little31', 'whocares31', 'Anthony_Little31@gmail.com', datetime.datetime(2017, 6, 1, 21, 12, 11), 10.0)
('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', datetime.datetime(2017, 5, 30, 17, 30, 22), 0.0)
('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', datetime.datetime(2017, 9, 9, 2, 30, 49), 10.0)
('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', datetime.datetime(2018, 1, 9, 17, 36, 49), 0.0)
('noah51', '18Oct1976#51', 'Noah51@gmail.com', datetime.datetime(2017, 12, 16, 22, 59, 46), 0.0)
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', datetime.datetime(2017, 10, 12, 5, 39, 20), 0.0)
('testin01', 'dlafjkgl', 'ladfdskl@gmail.com', datetime.date

In [74]:
create_delete_member = """
CREATE PROCEDURE delete_member(IN p_id VARCHAR(255))
BEGIN
    DELETE FROM members
    WHERE id = p_id;
END
"""

execute_sql(create_delete_member)

MySQL connection is closed


In [75]:
execute_sql("""CALL delete_member("testin01")""")

MySQL connection is closed


In [76]:
print_table("SELECT * FROM members")

('afeil', 'feil1988<3', 'Abdul.Feil@hotmail.com', datetime.datetime(2017, 4, 15, 12, 10, 13), 0.0)
('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', datetime.datetime(2018, 2, 6, 16, 48, 43), 0.0)
('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', datetime.datetime(2017, 12, 28, 5, 36, 50), 0.0)
('little31', 'whocares31', 'Anthony_Little31@gmail.com', datetime.datetime(2017, 6, 1, 21, 12, 11), 10.0)
('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', datetime.datetime(2017, 5, 30, 17, 30, 22), 0.0)
('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', datetime.datetime(2017, 9, 9, 2, 30, 49), 10.0)
('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', datetime.datetime(2018, 1, 9, 17, 36, 49), 0.0)
('noah51', '18Oct1976#51', 'Noah51@gmail.com', datetime.datetime(2017, 12, 16, 22, 59, 46), 0.0)
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', datetime.datetime(2017, 10, 12, 5, 39, 20), 0.0)
('wyattgreat', 'wyatt111', 'Wyatt_Wisozk2@gmail.com', dateti

In [78]:
create_update_member_password = """
CREATE PROCEDURE update_member_password(IN p_id VARCHAR(255), IN p_password VARCHAR(255))
BEGIN
    UPDATE members
    SET password = p_password
    WHERE id = p_id;
END
"""

execute_sql(create_update_member_password)

MySQL connection is closed


In [16]:
execute_sql("""CALL update_member_password("afeil", "new_password")""")

MySQL connection is closed


In [10]:
print_table("SELECT * FROM members")

('afeil', 'new_password', 'Abdul.Feil@hotmail.com', datetime.datetime(2017, 4, 15, 12, 10, 13), 0.0)
('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', datetime.datetime(2018, 2, 6, 16, 48, 43), 0.0)
('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', datetime.datetime(2017, 12, 28, 5, 36, 50), 0.0)
('little31', 'whocares31', 'Anthony_Little31@gmail.com', datetime.datetime(2017, 6, 1, 21, 12, 11), 10.0)
('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', datetime.datetime(2017, 5, 30, 17, 30, 22), 0.0)
('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', datetime.datetime(2017, 9, 9, 2, 30, 49), 10.0)
('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', datetime.datetime(2018, 1, 9, 17, 36, 49), 0.0)
('noah51', '18Oct1976#51', 'Noah51@gmail.com', datetime.datetime(2017, 12, 16, 22, 59, 46), 0.0)
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', datetime.datetime(2017, 10, 12, 5, 39, 20), 0.0)
('wyattgreat', 'wyatt111', 'Wyatt_Wisozk2@gmail.com', date

In [11]:
create_update_member_email = """
CREATE PROCEDURE update_member_email(IN p_id VARCHAR(255), IN p_email VARCHAR(255))
BEGIN
    UPDATE members
    SET email = p_email
    WHERE id = p_id;
END
"""

execute_sql(create_update_member_email)

MySQL connection is closed


In [14]:
execute_sql("""CALL update_member_email("afeil", "new_email@gmail.com")""")

MySQL connection is closed


In [17]:
print_table("SELECT * FROM members")

('afeil', 'new_password', 'new_email@gmail.com', datetime.datetime(2017, 4, 15, 12, 10, 13), 0.0)
('amely_18', 'loseweightin18', 'Amely.Bauch91@yahoo.com', datetime.datetime(2018, 2, 6, 16, 48, 43), 0.0)
('bbahringer', 'iambeau17', 'Beaulah_Bahringer@yahoo.com', datetime.datetime(2017, 12, 28, 5, 36, 50), 0.0)
('little31', 'whocares31', 'Anthony_Little31@gmail.com', datetime.datetime(2017, 6, 1, 21, 12, 11), 10.0)
('macejkovic73', 'jadajeda12', 'Jada.Macejkovic73@gmail.com', datetime.datetime(2017, 5, 30, 17, 30, 22), 0.0)
('marvin1', 'if0909mar', 'Marvin_Schulist@gmail.com', datetime.datetime(2017, 9, 9, 2, 30, 49), 10.0)
('nitzsche77', 'bret77@#', 'Bret_Nitzsche77@gmail.com', datetime.datetime(2018, 1, 9, 17, 36, 49), 0.0)
('noah51', '18Oct1976#51', 'Noah51@gmail.com', datetime.datetime(2017, 12, 16, 22, 59, 46), 0.0)
('oreillys', 'reallycool#1', 'Martine_OReilly@yahoo.com', datetime.datetime(2017, 10, 12, 5, 39, 20), 0.0)
('wyattgreat', 'wyatt111', 'Wyatt_Wisozk2@gmail.com', datetim

In [20]:
create_make_booking = """
CREATE PROCEDURE make_booking(IN p_room_id VARCHAR(255), IN p_booked_date DATE, IN p_booked_time TIME, IN p_member_id VARCHAR(255))
BEGIN
    DECLARE v_price FLOAT(5,2);
    DECLARE v_payment_due FLOAT(4,2);
    SELECT price INTO v_price FROM rooms WHERE id = p_room_id;
    INSERT INTO bookings (room_id, booked_date, booked_time, member_id) VALUES (p_room_id,p_booked_date, p_booked_time, p_member_id);
    SELECT payment_due INTO v_payment_due FROM members WHERE id = p_member_id;
    UPDATE members SET payment_due = (v_payment_due + v_price) WHERE id = p_member_id;
END
"""

execute_sql(create_make_booking)

MySQL connection is closed


In [22]:
create_update_payment = """
CREATE PROCEDURE update_payment(IN p_id INT)
BEGIN
    DECLARE v_member_id VARCHAR(255);
    DECLARE v_payment_due FLOAT(4,2);
    DECLARE v_price FLOAT(5,2);
    UPDATE bookings SET payment_status = 'Paid' WHERE id = p_id;
    SELECT member_id INTO v_member_id FROM member_bookings WHERE id = p_id;
    SELECT price INTO v_price FROM member_bookings WHERE id = p_id;
    SELECT payment_due INTO v_payment_due FROM members WHERE id = v_member_id;
    UPDATE members SET payment_due = (v_payment_due - v_price) WHERE id = v_member_id;
END
"""

execute_sql(create_update_payment)

MySQL connection is closed


In [28]:
create_view_bookings = """
CREATE PROCEDURE view_bookings(IN p_id VARCHAR(255))
BEGIN
    SELECT * FROM member_bookings WHERE member_id = p_id;
END
"""

execute_sql(create_view_bookings)

MySQL connection is closed


In [7]:
print_table(""" CALL view_bookings("marvin1")""")

(5, 'MPF2', 'Multi Purpose Field', datetime.date(2018, 3, 2), datetime.timedelta(seconds=39600), 'marvin1', datetime.datetime(2018, 3, 1, 16, 13, 45), 60.0, 'Paid')
(6, 'B1', 'Badminton Court', datetime.date(2018, 3, 28), datetime.timedelta(seconds=57600), 'marvin1', datetime.datetime(2018, 3, 23, 22, 46, 36), 8.0, 'Paid')
(9, 'T1', 'Tennis Court', datetime.date(2018, 5, 25), datetime.timedelta(seconds=36000), 'marvin1', datetime.datetime(2018, 5, 21, 11, 20, 46), 10.0, 'Unpaid')


In [36]:
create_search_room = """
CREATE PROCEDURE search_room(IN p_room_type VARCHAR(255), IN p_booked_date DATE, IN p_booked_time TIME)
BEGIN
   SELECT * FROM rooms WHERE id NOT IN 
        (SELECT room_id FROM member_bookings
        WHERE booked_date = p_booked_date  AND 
        booked_time = p_booked_time AND 
        payment_status != 'Cancelled')
        AND room_type = p_room_type;
END

"""
execute_sql(create_search_room)

MySQL connection is closed


In [37]:
print_table(""" CALL search_room("Tennis Court", "2017-12-26", "13:00:00") """)

('T1', 'Tennis Court', 10.0)
('T2', 'Tennis Court', 10.0)


In [64]:
create_cancel_booking = """
CREATE PROCEDURE cancel_booking(IN p_booking_id INT, OUT p_message VARCHAR(255))
BEGIN 
    DECLARE v_cancellation INT;
    DECLARE v_member_id VARCHAR(255);
    DECLARE v_payment_status VARCHAR(255);
    DECLARE v_booked_date DATE;
    DECLARE v_price FLOAT(5,2);
    DECLARE v_payment_due FLOAT(4,2);
    SET v_cancellation = 0;
    SELECT member_id, booked_date, price, payment_status INTO v_member_id, v_booked_date, v_price, v_payment_status FROM
    member_bookings 
    WHERE
    id =p_booking_id;
    SELECT payment_due INTO v_payment_due FROM members WHERE id = v_member_id;
    IF curdate() >= v_booked_date THEN
        SELECT 'Cancellation cannot be done on/after the booked date' INTO p_message;
    ELSEIF v_payment_status = 'Cancelled' OR v_payment_status = 'Paid' THEN
        SELECT 'Booking has already been cacelled or paid' INTO p_message;
    ELSE
        UPDATE bookings SET payment_status = 'Cancelled' WHERE id = p_booking_id;
        SET v_payment_due = v_payment_due - v_price;
        SET v_cancellation = check_cancellation(p_booking_id);
        IF v_cancellation >=2 THEN
            SET v_payment_due = v_payment_due + 10.0;
        END IF;
        UPDATE members SET payment_due = v_payment_due WHERE id = v_member_id;
        SELECT 'Booking Cancelled' INTO p_message;
    END IF;
END
"""
execute_sql(create_cancel_booking)

MySQL connection is closed


## Triggers

In [67]:
create_trigger_payment_check = """
CREATE TRIGGER payment_check BEFORE DELETE ON members FOR EACH ROW
BEGIN
    DECLARE v_payment_due FLOAT(4,2);
    SELECT payment_due INTO v_payment_due FROM members WHERE id = OLD.id;
    IF v_payment_due > 0 THEN
        INSERT INTO pending_terminations (id, email, payment_due) VALUES (OLD.id, OLD.email, OLD.payment_due);
    END IF;
END
"""
execute_sql(create_trigger_payment_check)

MySQL connection is closed


## Stored Function

In [81]:
create_function_check_cancellation = """
CREATE FUNCTION check_cancellation(p_booking_id INT) RETURNS INT 
DETERMINISTIC
BEGIN
    DECLARE v_done INT;
    DECLARE v_cancellation INT;
    DECLARE v_current_payment_status VARCHAR(255);
        
    
    DECLARE cur CURSOR FOR
        SELECT payment_status FROM bookings 
        WHERE 
        member_id = (SELECT member_id FROM bookings WHERE id = p_booking_id) 
        ORDER BY daytime_of_booking DESC;
    
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET v_done = 1;
    
    SET v_done = 0;
    SET v_cancellation = 0;
    
    OPEN cur;
    
    cancellation_loop: LOOP
        FETCH cur INTO v_current_payment_status;
        IF v_current_payment_status != "Cancelled" OR v_done = 1 THEN LEAVE cancellation_loop;
        ELSEIF v_done =1 THEN LEAVE cancellation_loop;
        ELSE SET v_cancellation = v_cancellation +1;
        END IF;
    END LOOP;
    CLOSE cur;
    
    RETURN v_cancellation;
END

"""

execute_sql(create_function_check_cancellation)

MySQL connection is closed
